In [1]:
import sys
sys.path.append('../')

import trimesh
import pyvista as pv
from pyvista import PolyData, Plotter
import numpy as np
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume
# from irregular_object_packing.mesh.utils import print_mesh_info
# from irregular_object_packing.packing import initialize, nlc_optimisation as nlc, chordal_axis_transform as cat
# from irregular_object_packing.packing.growth_based_optimisation import SimSettings, Optimizer
# from irregular_object_packing.packing.plots import generate_tinted_colors
from importlib import reload

In [2]:
from irregular_object_packing.packing.optimizer import default_optimizer_config


DATA_FOLDER = "../data/mesh/"

optimizer = default_optimizer_config(N=10, mesh_dir=DATA_FOLDER)
optimizer.setup()
optimizer.resample_meshes(0.1)
plotter = Plotter(shape=(1, 2), border=False)
plotter.subplot(0, 0)
plotter.add_mesh(optimizer.shape0, color="red")
plotter.isometric_view()
camera = plotter.camera
plotter.subplot(0, 1)
plotter.add_mesh(optimizer.shape0.scale(0.1**(1/3)), color="red")
plotter.camera = camera
plotter.save_graphic("renders/packing-init-shrink.pdf")
plotter.show()


Widget(value="<iframe src='http://localhost:55167/index.html?ui=P_0x156259d80_0&reconnect=auto' style='width: …

In [3]:
plotter = Plotter(shape=(1, 2), border=False)
plotter.subplot(0, 0)
plotter.add_mesh(optimizer.shape0, color="red", show_edges=True)
plotter.isometric_view()
camera = plotter.camera
plotter.subplot(0, 1)
plotter.add_mesh(optimizer.shape, color="red", show_edges=True)
plotter.camera = camera
plotter.save_graphic("renders/packing-init-resample.pdf")
plotter.show()

Widget(value="<iframe src='http://localhost:55167/index.html?ui=P_0x1520ded40_1&reconnect=auto' style='width: …

In [4]:
plotter = optimizer.plotter.plot_objects()
plotter.open_gif("renders/packing-init.gif")
path = plotter.generate_orbital_path(n_points = 60)
plotter.orbit_on_path(path, write_frames=True)
plotter.close()


Widget(value="<iframe src='http://localhost:55167/index.html?ui=P_0x11078ea40_2&reconnect=auto' style='width: …

In [ ]:
from irregular_object_packing.cat.chordal_axis_transform import compute_cdt

meshes = optimizer.meshes_before(0)
tetmesh = compute_cdt(meshes+[optimizer.container])
plotter = Plotter()
for mesh in meshes:
    plotter.add_mesh(mesh, color="red", show_edges=True, line_width=2)
plotter.add_mesh_clip_plane(tetmesh, color="grey", show_edges=True, line_width=2, crinkle=0.5)
plotter.show()



In [5]:
optimizer.run(Ni=1)

scale	:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration	:   0%|          | 0/100 [00:00<?, ?it/s]

Object	:   0%|          | 0/11 [00:00<?, ?it/s]

In [6]:
meshes_before, meshes_after, cat_meshes, container = optimizer.recreate_scene(optimizer.idx)
plotter = Plotter(shape=(1, 2), border=False)
plotter.subplot(0, 0)
for mesh in meshes_before:
    plotter.add_mesh(mesh, color="red", )

for cat_mesh in cat_meshes:
    plotter.add_mesh(cat_mesh, color="yellow", opacity=0.3)

plotter.add_mesh(container, color="white", opacity=0.2)


plotter.subplot(0, 1)
for mesh in meshes_after:
    plotter.add_mesh(mesh, color="red", )

for cat_mesh in cat_meshes:
    plotter.add_mesh(cat_mesh, color="yellow", opacity=0.3)

plotter.add_mesh(container, color="white", opacity=0.2)

plotter.link_views()
plotter.isometric_view()
plotter.camera.zoom(1.2)
plotter.open_gif("renders/packing-first-step.gif")
path = plotter.generate_orbital_path(n_points = 60)
plotter.orbit_on_path(path, write_frames=True)
plotter.close()

In [28]:
optimizer.run(Ni=300)

scale	:  56%|#####5    | 5/9 [00:00<?, ?it/s]

Iteration	:  41%|####1     | 41/100 [00:00<?, ?it/s]

Object	:   0%|          | 0/11 [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [30]:
optimizer.plotter.generate_gif("renders/packing.gif")

In [22]:

plotter = Plotter(shape=(3, 1), border=False)
plotter.subplot(0, 0)
idx = 0 # first scale
meshes_before, meshes_after, cat_meshes, container = optimizer.recreate_scene(idx)
for mesh in meshes_before:
    plotter.add_mesh(mesh, color="red", )

plotter.add_mesh(container, color="white", opacity=0.2)

idx = 4 # first scale boundary
plotter.subplot(1, 0)
meshes_before, meshes_after, cat_meshes, container = optimizer.recreate_scene(idx)
for mesh in meshes_after:
    plotter.add_mesh(mesh, color="red", )
plotter.add_mesh(container, color="white", opacity=0.2)

idx = 16 # second scale
plotter.subplot(2, 0)
meshes_before, meshes_after, cat_meshes, container = optimizer.recreate_scene(idx)
for mesh in meshes_after:
    plotter.add_mesh(mesh, color="red", )
plotter.add_mesh(container, color="white", opacity=0.2)

plotter.link_views()
plotter.isometric_view()
plotter.camera.zoom(1.3)
plotter.save_graphic("renders/packing-scale_steps123_vert.pdf")
plotter.show()

Widget(value="<iframe src='http://localhost:55167/index.html?ui=P_0x2812c0ac0_15&reconnect=auto' style='width:…

In [24]:

plotter = Plotter(shape=(3, 1), border=False)
plotter.subplot(0, 0)
idx = 28 # first scale
meshes_before, meshes_after, cat_meshes, container = optimizer.recreate_scene(idx)
for mesh in meshes_after:
    plotter.add_mesh(mesh, color="red", )

plotter.add_mesh(container, color="white", opacity=0.2)

idx = 37 # first scale boundary
plotter.subplot(1, 0)
meshes_before, meshes_after, cat_meshes, container = optimizer.recreate_scene(idx)
for mesh in meshes_after:
    plotter.add_mesh(mesh, color="red", )
plotter.add_mesh(container, color="white", opacity=0.2)

idx = 58 # second scale
plotter.subplot(2, 0)
meshes_before, meshes_after, cat_meshes, container = optimizer.recreate_scene(idx)
for mesh in meshes_after:
    plotter.add_mesh(mesh, color="red", )
plotter.add_mesh(container, color="white", opacity=0.2)

plotter.link_views()
plotter.isometric_view()
plotter.camera.zoom(1.3)
plotter.save_graphic("renders/packing-scale_steps456.pdf")
plotter.show()

Widget(value="<iframe src='http://localhost:55167/index.html?ui=P_0x30c6939d0_17&reconnect=auto' style='width:…

In [27]:
from irregular_object_packing.cat.chordal_axis_transform import compute_cdt

meshes = optimizer.meshes_before(0)
tetmesh = compute_cdt(meshes+[optimizer.container])
plotter = Plotter()
for mesh in meshes:
    plotter.add_mesh(mesh, color="red", show_edges=True, line_width=1)
plotter.add_mesh_clip_plane(tetmesh, color="grey", show_edges=True, edge_color="grey", line_width=2, crinkle=0.5)
plotter.view_vector((1,1,0))
plotter.show()

Widget(value="<iframe src='http://localhost:55167/index.html?ui=P_0x3921997e0_20&reconnect=auto' style='width:…